In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('../data/dummy.csv')
print(data.head())

df = data[data.ticker=='A']
print(df.shape)

df.name = 'Event_Health_Target_IT'
df = df.iloc[:, 3:]
df.head()

       sector ticker           target_sector    height     width  \
0  Healthcare      A  Information Technology  0.953058  0.832804   
1  Healthcare      A  Information Technology  0.891118  0.252365   
2  Healthcare      A  Information Technology  0.108774  0.178169   
3  Healthcare      A  Information Technology  0.949999  0.423031   
4  Healthcare      A  Information Technology  0.896733  0.191954   

   gap_from_previous  target_sector_price  sector_price  
0           0.395561             0.162064      0.244326  
1           0.679399             0.549430      0.959828  
2           0.114399             0.880535      0.667235  
3           0.165649             0.364328      0.834916  
4           0.421091             0.903926      0.528623  
(10000, 8)


,height,width,gap_from_previous,target_sector_price,sector_price
0,0.953058,0.832804,0.395561,0.162064,0.244326
1,0.891118,0.252365,0.679399,0.549430,0.959828
2,0.108774,0.178169,0.114399,0.880535,0.667235
3,0.949999,0.423031,0.165649,0.364328,0.834916
4,0.896733,0.191954,0.421091,0.903926,0.528623


In [3]:
X = df.iloc[:, [0,1,2]]
y = df.iloc[:, 3]
print(y.head())
X.head()

0    0.162064
1    0.549430
2    0.880535
3    0.364328
4    0.903926
Name: target_sector_price, dtype: float64


,height,width,gap_from_previous
0,0.953058,0.832804,0.395561
1,0.891118,0.252365,0.679399
2,0.108774,0.178169,0.114399
3,0.949999,0.423031,0.165649
4,0.896733,0.191954,0.421091


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu

In [5]:
# data = input_data.read_data_sets('data/fashion',one_hot=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# print("Training set (images) shape: {shape}".format(shape=data.train.images.shape))
# print("Training set (labels) shape: {shape}".format(shape=data.train.labels.shape))

# # Shapes of test set
# print("Test set (images) shape: {shape}".format(shape=data.test.images.shape))
# print("Test set (labels) shape: {shape}".format(shape=data.test.labels.shape))
print("Training set (images) shape: {shape}".format(shape=X_train.shape))
print("Training set (labels) shape: {shape}".format(shape=y_train.shape))

# Shapes of test set
print("Test set (images) shape: {shape}".format(shape=X_test.shape))
print("Test set (labels) shape: {shape}".format(shape=y_test.shape))

Training set (images) shape: (8000, 3)
Training set (labels) shape: (8000,)
Test set (images) shape: (2000, 3)
Test set (labels) shape: (2000,)


In [7]:
# Create dictionary of target classes
label_dict = {
 0: 'T-shirt/top',
 1: 'Trouser',
 2: 'Pullover',
 3: 'Dress',
 4: 'Coat',
 5: 'Sandal',
 6: 'Shirt',
 7: 'Sneaker',
 8: 'Bag',
 9: 'Ankle boot',
}

In [8]:
X_train = np.reshape(X_train.values, (-1,200,3,1))

In [9]:
X_train.shape

(40, 200, 3, 1)

In [11]:
# plt.figure(figsize=[5,5])

# # Display the first image in training data
# plt.subplot(121)
# curr_img = np.reshape(X_train[0], (200,3))
# curr_lbl = y_train[0]
# plt.imshow(curr_img, cmap='gray')
# plt.title("(Label: " + str(label_dict[curr_lbl]) + ")")

# # Display the first image in testing data
# plt.subplot(122)
# curr_img = np.reshape(data.test.images[0], (28,28))
# curr_lbl = np.argmax(data.test.labels[0,:])
# plt.imshow(curr_img, cmap='gray')
# plt.title("(Label: " + str(label_dict[curr_lbl]) + ")")

In [12]:
# train_X = data.train.images.reshape(-1, 28, 28, 1)
# test_X = data.test.images.reshape(-1,28,28,1)
# train_X.shape, test_X.shape

AttributeError: 'DataFrame' object has no attribute 'train'

In [29]:
train_y = data.train.labels
test_y = data.test.labels
train_y.shape, test_y.shape

((55000, 10), (10000, 10))

In [19]:
training_iters = 2 
learning_rate = 0.01 
batch_size = 128

# MNIST data input (img shape: 28*28)
n_input = 28

# MNIST total classes (0-9 digits)
n_classes = 10

In [17]:
#both placeholders are of type float
x = tf.placeholder("float", [None, 200,3,1])
y = tf.placeholder("float", [None, 8000])

In [19]:
def conv1d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv1d(x, W, stride=1, padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x) 

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME')

In [14]:
weights = {
    'wc1': tf.get_variable('W0', shape=(3,3,1,32), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc2': tf.get_variable('W1', shape=(3,3,32,64), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc3': tf.get_variable('W2', shape=(3,3,64,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'wd1': tf.get_variable('W3', shape=(4*4*128,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'out': tf.get_variable('W6', shape=(128,8000), initializer=tf.contrib.layers.xavier_initializer()), 
}
biases = {
    'bc1': tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'bc2': tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
    'bc3': tf.get_variable('B2', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'bd1': tf.get_variable('B3', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('B4', shape=(8000), initializer=tf.contrib.layers.xavier_initializer()),
}

In [15]:
def conv_net(x, weights, biases):  

    # here we call the conv2d function we had defined above and pass the input image x, weights wc1 and bias bc1.
    conv1 = conv1d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 14*14 matrix.
    conv1 = maxpool1d(conv1, k=2)

    # Convolution Layer
    # here we call the conv2d function we had defined above and pass the input image x, weights wc2 and bias bc2.
    conv2 = conv1d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 7*7 matrix.
    conv2 = maxpool1d(conv2, k=2)

    conv3 = conv1d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 4*4.
    conv3 = maxpool1d(conv3, k=2)


    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [20]:
pred = conv_net(x, weights, biases)

cost = tf.losses.mean_squared_error(labels=y_train, predictions=pred)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

ValueError: Shape must be rank 4 but is rank 5 for 'conv1d/Conv2D' (op: 'Conv2D') with input shapes: [?,1,200,3,1], [1,3,3,1,32].

In [39]:
#Here you check whether the index of the maximum value of the predicted image is equal to the actual labelled image. and both will be a column vector.
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

#calculate accuracy across all the given images and average them out. 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [40]:
init = tf.global_variables_initializer()

In [42]:
with tf.Session() as sess:
    sess.run(init) 
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    for i in range(2):
        for batch in range(len(train_X)//batch_size):
            batch_x = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
            batch_y = train_y[batch*batch_size:min((batch+1)*batch_size,len(train_y))]    
            # Run optimization op (backprop).
                # Calculate batch loss and accuracy
            opt = sess.run(optimizer, feed_dict={x: batch_x,
                                                              y: batch_y})
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y})
        print("Iter " + str(i) + ", Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
        print("Optimization Finished!")

        # Calculate accuracy for all 10000 mnist test images
        test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={x: test_X,y : test_y})
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy.append(acc)
        test_accuracy.append(test_acc)
        print("Testing Accuracy:","{:.5f}".format(test_acc))
    summary_writer.close()

Iter 0, Loss= 0.010153, Training Accuracy= 1.00000
Optimization Finished!
Testing Accuracy: 0.97520
Iter 1, Loss= 0.007809, Training Accuracy= 1.00000
Optimization Finished!
Testing Accuracy: 0.98330
